In [2]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle
import numpy as np

2025-08-17 20:29:30.047504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-17 20:29:30.170569: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-17 20:29:31.224123: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [3]:
import sys
sys.path.append('../../../../')

## load

In [4]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-08-17 20:29:34.316716: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test', 'coordinates_train', 'coordinates_validation', 'coordinates_test'])

In [6]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

## debug pairwise transition

In [7]:
from model import build_model, create_masked_inputs

In [8]:
model = build_model(features_mlp_hidden_sizes=[[256, 256]],
            features_mlp_dropout_rate=0.2,
            output_mlp_hidden_sizes=[[256, 256]], output_mlp_dropout_rate=0.2,
            attention_mlp_hidden_sizes=[[256, 256]], attention_mlp_dropout_rate=0.2,
            activation='relu', input_shape=(10, 9), max_number_of_patches=10, attention_dimension=256,
            gaussian_xrange=(0, 200),
            pairs_channel_dimension=256, num_heads=8, use_pair_bias=True)

initializing projection for skip connection: 11 -> 256


In [9]:
batch_size = 7

In [13]:
model([input_data[:batch_size], coordinates[:batch_size], size_value[:batch_size], n_patches_hot_encoded_value[:batch_size]])

<tf.Tensor: shape=(7, 1), dtype=float32, numpy=
array([[0.71467316],
       [0.6878214 ],
       [0.7044209 ],
       [0.6882016 ],
       [0.6813201 ],
       [0.6741993 ],
       [0.70242476]], dtype=float32)>